# A Quantum Pipeline for the regression test case selection problem
The aim of this project is to exploit the capabilities of quantum algoritms and quantum computer to optimize the process of test case selection for regression testing purposes.

In [1]:
#this cell contains all the imports needed by the pipeline

## The pipeline dataset
The dataset to be used contains important information about 11 programs chose from the open-source SIR. 